## unsupervise learning on EN_dataset


In [131]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import spacy
nlp = spacy.load('en_core_web_sm')


In [132]:
df = pd.read_csv('ChinaDaily_EN_cleaned.csv')


In [133]:
df.head()

,Unnamed: 0,level_0,column_1,column_2,content,publishtime,source,title,url
0,0,0,Business,"100 cities, counties and companies",The artificial intelligence sector is expected...,2018-10-15 09:33,"QIU QUANLIN,CHINA DAILY",AI plays crucial role in industrial upgrading,http://www.chinadaily.com.cn/a/201810/15/WS5bc...
1,1,1,Business,"100 cities, counties and companies",The 124th session of China's largest trade eve...,2018-10-15 09:21,"QIU QUANLIN IN GUANGZHOU,CHINA DAILY",Guangzhou cements position as key trading hub,http://www.chinadaily.com.cn/a/201810/15/WS5bc...
2,2,2,Business,"100 cities, counties and companies","Sixteen years ago, leaving his home village of...",2018-10-16 09:33,"LI LEI, SHI BAOYIN AND QI XIN IN LANKAO, HENA...",Returning workers succeed at home,http://www.chinadaily.com.cn/a/201810/16/WS5bc...
3,3,3,Business,"100 cities, counties and companies","Pudong - literally ""east of Huangpu River"" - i...",2018-10-17 09:16,"WANG YING AND SHI JING IN SHANGHAI,CHINA DAILY",How Pudong rose from the shadows,http://www.chinadaily.com.cn/a/201810/17/WS5bc...
4,4,4,Business,"100 cities, counties and companies",Farmers in the Tibet autonomous region's Nying...,2018-10-19 09:35,"PALDEN NYIMA IN MANLING, TIBET,CHINA DAILY",Herbs grow Tibetan farmers' fortunes,http://www.chinadaily.com.cn/a/201810/19/WS5bc...


In [134]:
from sklearn.feature_extraction.text import CountVectorizer

In [135]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [136]:
dtm = cv.fit_transform(df['content'])

In [137]:
from sklearn.decomposition import LatentDirichletAllocation

In [138]:
LDA = LatentDirichletAllocation(n_components=5,random_state=42)

In [139]:
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=5, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [140]:
len(cv.get_feature_names())

67142

In [141]:
import random

In [142]:
for i in range(10):
    random_word_id = random.randint(0,13449)
    print(cv.get_feature_names()[random_word_id])

aerotropolis
brandon
bmce
4s
capitalize
barns
114
catalyzes
bingsong
adamant


In [143]:
len(LDA.components_)

5

In [144]:
len(LDA.components_[0])

67142

In [145]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-40:]])
    print('\n')

THE TOP 10 WORDS FOR TOPIC #0
['shanghai', 'based', 'group', 'internet', 'years', 'economy', 'total', 'beijing', '2017', 'research', 'sector', 'bank', 'high', 'global', 'government', 'country', 'report', 'services', 'products', 'investment', 'data', 'business', 'financial', 'development', 'technology', 'industry', 'million', 'chinese', 'new', 'company', 'growth', 'companies', 'according', 'yuan', 'billion', 'market', 'year', 'percent', 'said', 'china']


THE TOP 10 WORDS FOR TOPIC #1
['ancient', 'million', 'time', 'river', 'cpc', 'culture', 'protection', 'including', 'public', 'rural', 'county', 'new', 'areas', 'region', 'village', 'poverty', 'work', 'area', 'government', 'art', 'exhibition', 'committee', 'museum', 'beijing', 'water', 'central', 'according', 'year', 'years', 'city', 'cultural', 'local', 'chinese', 'party', '000', 'national', 'province', 'people', 'china', 'said']


THE TOP 10 WORDS FOR TOPIC #2
['police', 'zhang', 'court', 'million', 'public', 'wang', 'just', 'health',

### Attaching Discovered Topic Labels to Original Articles

In [146]:
dtm

<57225x67142 sparse matrix of type '<class 'numpy.int64'>'
	with 8315746 stored elements in Compressed Sparse Row format>

In [147]:
dtm.shape

(57225, 67142)

In [148]:
len(df)

57225

In [149]:
topic_results = LDA.transform(dtm)

In [150]:
topic_results.shape

(57225, 5)

In [151]:
topic_results[2]

array([3.81930831e-01, 4.15380073e-01, 1.97262247e-01, 5.09297452e-03,
       3.33874990e-04])

In [152]:
topic_results[5].round(2)

array([0.21, 0.  , 0.11, 0.64, 0.04])

In [153]:
topic_results.argmax(axis=1)

array([0, 0, 1, ..., 2, 2, 3])

In [154]:
df['LDA_Topic'] = topic_results.argmax(axis=1)

In [155]:
df.head()

,Unnamed: 0,level_0,column_1,column_2,content,publishtime,source,title,url,LDA_Topic
0,0,0,Business,"100 cities, counties and companies",The artificial intelligence sector is expected...,2018-10-15 09:33,"QIU QUANLIN,CHINA DAILY",AI plays crucial role in industrial upgrading,http://www.chinadaily.com.cn/a/201810/15/WS5bc...,0
1,1,1,Business,"100 cities, counties and companies",The 124th session of China's largest trade eve...,2018-10-15 09:21,"QIU QUANLIN IN GUANGZHOU,CHINA DAILY",Guangzhou cements position as key trading hub,http://www.chinadaily.com.cn/a/201810/15/WS5bc...,0
2,2,2,Business,"100 cities, counties and companies","Sixteen years ago, leaving his home village of...",2018-10-16 09:33,"LI LEI, SHI BAOYIN AND QI XIN IN LANKAO, HENA...",Returning workers succeed at home,http://www.chinadaily.com.cn/a/201810/16/WS5bc...,1
3,3,3,Business,"100 cities, counties and companies","Pudong - literally ""east of Huangpu River"" - i...",2018-10-17 09:16,"WANG YING AND SHI JING IN SHANGHAI,CHINA DAILY",How Pudong rose from the shadows,http://www.chinadaily.com.cn/a/201810/17/WS5bc...,0
4,4,4,Business,"100 cities, counties and companies",Farmers in the Tibet autonomous region's Nying...,2018-10-19 09:35,"PALDEN NYIMA IN MANLING, TIBET,CHINA DAILY",Herbs grow Tibetan farmers' fortunes,http://www.chinadaily.com.cn/a/201810/19/WS5bc...,1


In [156]:
#based on the results


Topic0 = 'business'
Topic1 = 'lifestyle'
Topic2 = 'education'
Topic3 = 'tech'
Topic4 = 'politics'

In [157]:
def relabel(num):
    if num == 0:
        return Topic0
    elif num == 1:
        return Topic1
    elif num == 2:
        return Topic2
    elif num == 3:
        return Topic3
    else:
        return Topic4

In [158]:
df['LDA_Topic'] = df['LDA_Topic'].apply(relabel)

In [159]:
df[df['overall label'] == df['LDA_Topic']]

KeyError: 'overall label'

In [98]:
#the correct percentage is 
len(df[df['overall label'] == df['LDA_Topic']])/len(df)

0.21994884910485935

In [101]:
df_temp = df[df['business'] == 0]

In [104]:
(df_temp['LDA_Topic'] == 'business').value_counts()

False    644
True     288
Name: LDA_Topic, dtype: int64

## using NMF algorithm 

In [160]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [161]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [162]:
dtm = tfidf.fit_transform(df['content'])

In [163]:
from sklearn.decomposition import NMF

In [164]:
nmf_model = NMF(n_components=5,random_state=42)

In [165]:
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=5, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [166]:
nmf_model.components_

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.16656020e-05, 9.27827158e-06, 0.00000000e+00],
       [4.14098926e-04, 8.10667464e-02, 0.00000000e+00, ...,
        1.10141603e-04, 1.25851542e-04, 5.75494133e-05],
       [9.26498174e-03, 4.09160252e-01, 0.00000000e+00, ...,
        1.79029583e-05, 5.36633659e-05, 2.78263070e-04],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.02583641e-02, 2.79239948e-01, 5.66004858e-03, ...,
        1.14894486e-05, 3.61411227e-05, 1.28445823e-04]])

In [167]:
len(nmf_model.components_)

5

In [168]:
single_topic = nmf_model.components_[0]

In [169]:
single_topic.argsort()

array([    0, 39264, 39265, ..., 14073, 11348, 58909])

In [170]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['xi', 'world', 'development', 'chinese', 'global', 'initiative', 'international', 'belt', 'economic', 'road', 'cooperation', 'said', 'countries', 'china', 'trade']


THE TOP 15 WORDS FOR TOPIC #1
['quarter', 'investment', 'sales', 'report', 'million', 'financial', 'china', 'market', 'bank', 'trillion', 'growth', 'billion', 'yuan', 'year', 'percent']


THE TOP 15 WORDS FOR TOPIC #2
['government', 'services', 'million', '000', 'development', 'city', 'according', 'industry', 'internet', 'companies', 'new', 'technology', 'company', 'china', 'said']


THE TOP 15 WORDS FOR TOPIC #3
['secretary', 'poverty', 'military', 'said', 'jinping', 'national', 'congress', 'communist', 'people', 'political', 'central', 'committee', 'cpc', 'party', 'xi']


THE TOP 15 WORDS FOR TOPIC #4
['school', 'year', 'china', 'people', 'music', 'culture', 'students', 'festival', 'exhibition', 'museum', 'film', 'cultural', 'art', 'says', 'chinese']




In [171]:
Topic0 = 'politics'
Topic1 = 'business'
Topic2 = 'tech'
Topic3 = 'politics'
Topic4 = 'education_lifestyle'

In [172]:
topic_results = nmf_model.transform(dtm)

In [173]:
df['NMF_Topic'] = topic_results.argmax(axis=1)

In [174]:
df['NMF_Topic'] = df['NMF_Topic'].apply(relabel)

In [175]:
df.head()

,Unnamed: 0,level_0,column_1,column_2,content,publishtime,source,title,url,LDA_Topic,NMF_Topic
0,0,0,Business,"100 cities, counties and companies",The artificial intelligence sector is expected...,2018-10-15 09:33,"QIU QUANLIN,CHINA DAILY",AI plays crucial role in industrial upgrading,http://www.chinadaily.com.cn/a/201810/15/WS5bc...,business,tech
1,1,1,Business,"100 cities, counties and companies",The 124th session of China's largest trade eve...,2018-10-15 09:21,"QIU QUANLIN IN GUANGZHOU,CHINA DAILY",Guangzhou cements position as key trading hub,http://www.chinadaily.com.cn/a/201810/15/WS5bc...,business,politics
2,2,2,Business,"100 cities, counties and companies","Sixteen years ago, leaving his home village of...",2018-10-16 09:33,"LI LEI, SHI BAOYIN AND QI XIN IN LANKAO, HENA...",Returning workers succeed at home,http://www.chinadaily.com.cn/a/201810/16/WS5bc...,lifestyle,tech
3,3,3,Business,"100 cities, counties and companies","Pudong - literally ""east of Huangpu River"" - i...",2018-10-17 09:16,"WANG YING AND SHI JING IN SHANGHAI,CHINA DAILY",How Pudong rose from the shadows,http://www.chinadaily.com.cn/a/201810/17/WS5bc...,business,tech
4,4,4,Business,"100 cities, counties and companies",Farmers in the Tibet autonomous region's Nying...,2018-10-19 09:35,"PALDEN NYIMA IN MANLING, TIBET,CHINA DAILY",Herbs grow Tibetan farmers' fortunes,http://www.chinadaily.com.cn/a/201810/19/WS5bc...,lifestyle,tech


In [176]:
#两种不同算法的标签吻合度
len(df[df['LDA_Topic'] == df['NMF_Topic']])/len(df)

0.345653123634775

In [181]:
df_temp = df[df['NMF_Topic'] == 'education_lifestyle']

In [182]:
dtm = tfidf.fit_transform(df_temp['content'])

In [183]:
nmf_model = NMF(n_components=2,random_state=42)

In [184]:
nmf_model.fit(dtm)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=2, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [185]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['education', 'children', 'new', 'university', 'school', 'people', 'festival', 'music', 'year', 'says', 'students', 'china', 'film', 'said', 'chinese']


THE TOP 15 WORDS FOR TOPIC #1
['relics', 'painting', 'beijing', 'culture', 'heritage', 'works', 'paintings', 'palace', 'artists', 'chinese', 'china', 'cultural', 'exhibition', 'art', 'museum']




In [194]:
Topic0 = 'education'
Topic1 = 'lifestyle'

In [195]:
topic_results = nmf_model.transform(dtm)

In [196]:
df_temp['NMF_Topic'] = topic_results.argmax(axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [197]:
df_temp['NMF_Topic'] = df_temp['NMF_Topic'].apply(relabel)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [198]:
df_temp.head()

,Unnamed: 0,level_0,column_1,column_2,content,publishtime,source,title,url,LDA_Topic,NMF_Topic
11,11,11,Business,"100 cities, counties and companies",A lack of scenic attractions has not stopped S...,2018-10-26 09:09,"FENG ZHIWEI AND ZHANG ZHOUXIANG IN CHANGSHA,C...",Red tourism pays off for Shaoshan,http://www.chinadaily.com.cn/a/201810/26/WS5bd...,lifestyle,education
22,22,22,Business,"100 cities, counties and companies","In a little over two decades, Malanshan has be...",2018-11-16 09:34,"CHENG SI AND FENG ZHIWEI IN CHANGSHA, HUNAN P...",From rural backwater to ideas factory,http://www.chinadaily.com.cn/a/201811/16/WS5be...,education,education
33,33,33,Business,"100 cities, counties and companies","For Chongming High School student Zhu Yuechen,...",2018-12-05 08:58,"XING YI IN SHANGHAI,CHINA DAILY",Trips take learning out of classroom,http://www.chinadaily.com.cn/a/201812/05/WS5c0...,education,education
43,43,43,Business,"100 cities, counties and companies",It's said that Empress Wu Zetian of the Tang D...,2018-12-20 09:16,"LI YINGXUE,CHINA DAILY",Success takes root in peony capital,http://www.chinadaily.com.cn/a/201812/20/WS5c1...,lifestyle,education
45,45,45,Business,2017 China-Russia Internet Media Foruma,As an integral part of the China-Russia Foru...,2017-09-14 18:41,"ZHANG XINGJIAN,CHINADAILY.COM.CN",Students ready for China-Russia • Youth Collab...,http://www.chinadaily.com.cn/a/201709/14/WS59b...,education,education


In [209]:
df[]

KeyboardInterrupt: 

In [ ]:
df.head()